## LangChain chains

In [ ]:
!pip install -q langchain langchain-openai python-dotenv

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

print("Model gotowy.")


Model gotowy.


### Prosty chain: prompt → model → wynik

In [2]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Definiujemy prompt
prompt = ChatPromptTemplate.from_messages([
    ("system", "Jesteś pomocnym asystentem."),
    ("user", "Streść w jednym zdaniu: {tekst}")
])

# Tworzymy chain: prompt → llm → parser
chain = prompt | llm | StrOutputParser()

# Test
wynik = chain.invoke({"tekst": "LangChain to biblioteka do pracy z dużymi modelami językowymi."})
print(wynik)


LangChain to biblioteka umożliwiająca efektywne wykorzystanie dużych modeli językowych w różnych aplikacjach.


### Sequential chain: dwa modele w sekwencji

In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 1) Chain: streszczenie (wejście: {tekst} → wyjście: str)
summary_chain = (
    ChatPromptTemplate.from_messages([
        ("system", "Streszcz poniższy tekst w 1–2 zdaniach po polsku."),
        ("user", "{tekst}")
    ])
    | llm
    | StrOutputParser()
)

# 2) Adapter: zamień str → {"tekst": str}, żeby podać do następnego promptu
to_dict = RunnableLambda(lambda s: {"tekst": s})

# 3) Chain: tłumaczenie streszczenia na francuski (wejście: {tekst} → wyjście: str)
translate_chain = (
    ChatPromptTemplate.from_messages([
        ("system", "Przetłumacz tekst na język francuski."),
        ("user", "{tekst}")
    ])
    | llm
    | StrOutputParser()
)

# 4) Prawdziwy sequential chain: summary → (wrap) → translate
sequential_chain = summary_chain | to_dict | translate_chain

# Test — jedno wywołanie całego łańcucha
input_text = "LangChain umożliwia tworzenie aplikacji AI poprzez łączenie modeli, promptów i narzędzi w spójne pipeline’y."
final_translation = sequential_chain.invoke({"tekst": input_text})

print("Tłumaczenie francuskie:\n", final_translation)



Streszczenie: LangChain pozwala na budowanie aplikacji AI poprzez integrację modeli, promptów i narzędzi w zorganizowane procesy robocze.
Tłumaczenie: LangChain permet de construire des applications d'IA en intégrant des modèles, des invites et des outils dans des processus de travail organisés.


### Branching chain: jedna odpowiedź, dwa przetworzenia

In [ ]:
from langchain_core.runnables import RunnableParallel

# Prompt do streszczenia
prompt_summary = ChatPromptTemplate.from_template("Streszcz: {tekst}")

# Prompt do sentymentu
prompt_sentiment = ChatPromptTemplate.from_template("Określ ton wypowiedzi: {tekst}")

# Dwa równoległe przetworzenia
branch_chain = RunnableParallel(
    summary=(prompt_summary | llm | StrOutputParser()),
    sentiment=(prompt_sentiment | llm | StrOutputParser())
)

text = "Jestem bardzo zadowolony z tego kursu, nauczyłem się dużo o LangChain!"
result = branch_chain.invoke({"tekst": text})

print(result)


### Prosty RAG chain

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
prosty chain: prompt → model → wynik
# Dokumenty
docs = [
    "LangChain to framework do pracy z dużymi modelami językowymi.",
    "Chains w LangChain to przepływy danych pomiędzy promptami, modelami i parserami.",
    "Retriever pozwala wyszukiwać informacje w bazie wektorowej."
]

# Split i embeddings
splitter = RecursiveCharacterTextSplitter(chunk_size=50, chunk_overlap=10)
splitted = splitter.split_text(" ".join(docs))

embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_texts(splitted, embedding=embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

# Prompt RAG
rag_prompt = ChatPromptTemplate.from_messages([
    ("system", "Odpowiedz tylko na podstawie KONTEKSTU:\n{context}"),
    ("user", "{question}")
])

def format_docs(docs):
    return "\n".join(d.page_content for d in docs)

from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
    | StrOutputParser()
)

print(rag_chain.invoke("Czym są chains w LangChain?"))
